### **Global Warming Animation**

##### Object: 1880년부터 2021년까지의 연평균 기온 편차를 보여주는 지구 온난화 애니메이션 제작 및 설명

In [1]:
### 0. Installing & Calling packages


!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install "basemap == 1.3.0b1" "basemap-data == 1.3.0b1"
!pip install pyproj==1.9.6
!pip install basemap
!pip install pillow

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
     |████████████████████████████████| 880 kB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 30.5 MB 393 kB/s eta 0:00:01
     |████████████████████████████████| 219 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 10.3 MB 16.6 MB/s eta 0:00:01
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37262 sha256=087961d9e5e15b142471f22f6da3888c755af4348bdce25bd0e8a46cfec7f7fc
  Stored in directory: /home/ydduri/.cache/pip/wheels/1f/1b/b5/54affbefc8a7e2bdf1da000fc576b8a1c91338f1f327a04f4c
Successfully built pyshp
  Attempting un

In [7]:
### 0. Installing & Calling packages


from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
%matplotlib inline
from netCDF4 import Dataset
import numpy as np
from PIL import Image
import datetime as dt

In [4]:
### 1. Loading the data


data = Dataset('gistemp1200_GHCNv4_ERSSTv5.nc')
print(data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: GISTEMP Surface Temperature Analysis
    institution: NASA Goddard Institute for Space Studies
    source: http://data.giss.nasa.gov/gistemp/
    Conventions: CF-1.6
    history: Created 2022-09-12 12:19:48 by SBBX_to_nc 2.0 - ILAND=1200, IOCEAN=NCDC/ER5, Base: 1951-1980
    dimensions(sizes): lat(90), lon(180), time(1712), nv(2)
    variables(dimensions): float32 lat(lat), float32 lon(lon), int32 time(time), int32 time_bnds(time, nv), int16 tempanomaly(time, lat, lon)
    groups: 


In [6]:
### 2. 데이터 구조 파악하기


# 기상자료를 구성하고 있는 여러 key들을 각각 살펴보자

print(data['time'])         # 매월의 대푯값, 대체로 중간값인 15일 경의 값
print('--------------------')
print(data['time_bnds'])    # 시간 간격을 나타내며, 여기서는 한 달
print('--------------------')
print(data['tempanomaly'])  # 온도 상승 편차(Base: 1951~1980을 기준으로 편차 계산)

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    long_name: time
    units: days since 1800-01-01 00:00:00
    bounds: time_bnds
unlimited dimensions: 
current shape = (1712,)
filling on, default _FillValue of -2147483647 used
--------------------
<class 'netCDF4._netCDF4.Variable'>
int32 time_bnds(time, nv)
unlimited dimensions: 
current shape = (1712, 2)
filling on, default _FillValue of -2147483647 used
--------------------
<class 'netCDF4._netCDF4.Variable'>
int16 tempanomaly(time, lat, lon)
    long_name: Surface temperature anomaly
    units: K
    scale_factor: 0.01
    cell_methods: time: mean
    _FillValue: 32767
unlimited dimensions: 
current shape = (1712, 90, 180)
filling on


In [8]:
### 3. time key와 실제 날짜 맞추기


print(data['time'][:])
x = dt.datetime(1800, 1, 1)
y = x + dt.timedelta(days = 29233)
print(y)

[29233 29264 29293 ... 81249 81279 81310]
1880-01-15 00:00:00


In [9]:
### 4. 1880년부터 2021년까지의 연평균 기온 편차 그림 연도별로 그리기


mp = Basemap(projection='robin',
             lon_0=0,                 # 경도 0도 기준으로 그림 그리기
             resolution='i')  


Lon = data['lon'][:]
Lat = data['lat'][:]
time = data['time'][:]
time_bnds = data['time_bnds'][:]
tempanomaly = data['tempanomaly'][:]


# 경도 및 위도 정보를 담고 있는 격자 구성
lon, lat = np.meshgrid(Lon, Lat)


# 경도 및 위도를 그림에 표기
x, y = mp(lon, lat)


# 각 연도에 접근
for year in range(0, 142):
    print(year)
    yearly_sum = tempanomaly[year*12, :, :] # 1월 자료 더해짐
    
    # 각 연도의 매월 자료에 접근
    for month in (1, 12):
        yearly_sum += tempanomaly[year*12 + month, :, :] # 2~12월 자료 더해짐
    
    yearly_temp_anomaly = np.squeeze(yearly_sum/12)

    c_scheme = mp.pcolor(x, y, yearly_temp_anomaly, cmap='jet')   
    mp.drawcoastlines()
    mp.drawcountries()
    mp.drawstates()
    
    cbar = mp.colorbar(c_scheme, location='right', pad='10%')
    cbar.set_label('$^o$C')

    plt.title('Global Temperature Anomaly Timelapse \nBaseline 1951-1980\n%d\nmade by outofweather' % (year+1880))

    plt.clim(-4, 4)

    # plt.savefig(r"drive/MyDrive/Colab Notebooks/global_warming_animation_%d.jpg" % (year+1880))

    plt.clf()

0


/tmp/ipykernel_2452/4186638798.py:35: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  c_scheme = mp.pcolor(x, y, yearly_temp_anomaly, cmap='jet')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141


<Figure size 432x288 with 0 Axes>

In [ ]:
### 5. 4에서 낱개로 그린 그림 플립북 애니메이션처럼 순서대로 넘어가면서 재생될 수 있도록 gif 파일로 합치기


# 이미지들이 더해질 프레임 생성
image_frames = []


# 연도 array 생성
years = np.arange(1880, 1880+142)


# 매 년도에 대해서 위 프레임에 이미지들을 차곡차곡 더하기
for year in years:
    new_frame = Image.open(r"drive/MyDrive/Colab Notebooks/global_warming_animation_%d.jpg" % (year))
    image_frames.append(new_frame)

image_frames[0].save('Global_Warming_1880_to_2021_GIF.gif',
                     format='GIF',
                     append_images=image_frames[1:],
                     save_all=True,
                     duration=100,
                     loop=0) # loop=0은 무한루프, 그 외 숫자는 지정한 만큼 반복


# 완성된 gif 파일을 내 컴퓨터에 다운로드
# files.download('Global_Warming_1880_to_2021_GIF.gif')